# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Problem Statement

The problem is to identify the subcategory and classify the question based on the group it belongs to.



## Learning Objectives

At the end of the experiment, you will be able to understand:

*   Beautiful Soup
*   Use NLTK package
*   Text Representation
*   Classification

## Dataset
Being able to classify the questions will be difficult in natural language processing. The dataset is taken from the TalentSprint aptitude questions which contains more than 20K questions.

## Description
This dataset has the following columns:
1. **Category:** Gives the high-level categorization of the question
2. **Sub-Category:** Determines the type of questions
3. **Article:** Gives the article name of the question
4. **Questions:** Questions are listed
5. **Answers:** Contains answers


### Grading = 20 Marks

In [ ]:
! wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Aptitude_Classification_data.csv
! wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Mentors_Test_Data.csv
    

In [ ]:
# Import Python Libraries
from bs4 import BeautifulSoup
import nltk
import re
import string
import warnings
import numpy as np
import pandas as pd
from collections import Counter
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

##   **Stage 1**:  Dataset Preparation

### 1 Mark -> Load the data set and prepare the data based on group allocation. 

Each group should consider their respective sub-categories as mentioned below:

> Team A = Groups 1, 4, 7, 10;   &nbsp; &nbsp; Sub-Category = Misspell words, Algebra, Percentages, Mathematical operations, Probability

> Team B = Groups 2, 5, 8, 11; &nbsp; &nbsp;   Sub-Category = Finding Errors, Ratio and Proportion, Logarithms, Time and Distance, Simple and Compound Interest

> Team C = Groups 3, 6, 9, 12;  &nbsp; &nbsp;  Sub-Category =  Synonyms and Antonyms, Time and Work, Permutations and Combinations, LCM and HCF, Profit and Loss


**Hint:** &nbsp; To access Sub-Categories from given Data, refer [link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html)

In [ ]:
# YOUR CODE HERE TO LOAD THE APTITUDE CLASSIFICATION DATASET & EXTRACT THE DATA BASED ON YOUR SUB-CATEGORIES
data = pd.read_csv("Aptitude_Classification_data.csv")
print(data.shape)
data.head()

(4631, 5)


,Category,Sub-Category,Article,Questions,Answers
0,Verbal,Misspell words,chapter 1,Which of the following is correct?\n\n\n\n\n,2
1,Quantitative,Time and Distance,Time and Distance - Model 05,Rohan leaves point A and reaches point B in 6 ...,2
2,Verbal,Finding Errors,44054,Read the sentence to find out whether there is...,2
3,Quantitative,Time and Work,tech mahindra_5th August,4 men can check exam papers in 8 days working ...,2
4,Quantitative,Permutations and Combinations,AX10DEPT01,"From 13 persons, how many ways of selection of...",2


In [ ]:
data.shape

(4631, 5)

In [ ]:
categories = [['Misspell words','Algebra', 'Percentages', 'Mathematical operations','Probability'],
            ['Finding Errors', 'Ratio and Proportion', 'Logarithms', 'Time and Distance', 'Simple and Compound Interest'],
            ['Synonyms and Antonyms','Time and Work', 'Permutations and Combinations', 'LCM and HCF', 'Profit and Loss']]

## **Stage 2:** Data Pre-Processing

### 3 Marks -> Clean and Transform the data into a specified format

*   Remove the rows of the Questions column which contains blank / NaN.


*   Few set of questions have HTML tags within the question.
  - You can use Beautiful Soup library to convert HTML into text (Refer **"Dealing with HTML"** section from this [link](https://www.nltk.org/book/ch03.html).)


*  Consider Question column as feature and Sub-category as target variable. Convert Sub-category into numerical.

*  Drop the unwanted columns


  **Hint:** Use Label Encoder for obtaining a numeric representation, refer to the [link](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). 

In [ ]:
print(data.isnull().sum())

In [ ]:
print("Shape of the data before dropping the na values",data.shape)
data.dropna(inplace=True)

Team_data = data[data['Sub-Category'].isin(category)]
print("Shape of the team data is:",Team_data.shape)
Team_data.head()

In [ ]:
# YOUR CODE HERE for BeatifulSoup

from bs4 import BeautifulSoup                                                                                                                                                                   

def cleanandTransform(data1):
  data1 = data1.drop(list(data1[data1["Questions"].isnull()].index)) 
  data1["Questions"] = [BeautifulSoup(text,"html.parser").get_text() for text in data1["Questions"]]
  return data1.drop(['Category','Article','Answers','Sub-Category'],axis=1)

features = cleanandTransform(Team_data)
print(features.shape)
features.head()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() # DO NOT CHANGE THIS LINE as we will be using for the Test evaluation.

# YOUR CODE HERE for Fit label encoder and return encoded labels

targets = le.fit(Team_data['Sub-Category'])
targets = le.transform(Team_data['Sub-Category'])
print("Shape of the Targets is: ",targets.shape)

## **Stage 3:** Text representation using Bag of Words (BOW)

### 3 Marks -> a) Get valid words from all questions & add them to a list.


Treat each question as a separate document and get the list of words using the following:
1.   Split the sentence into words

2.   Remove Stop words. Use NLTK packages for getting the Stop words.

3.   Replace proper names with "name" 
  - Example: "Rahul" -> "name"
       
4.   Remove the single white space character (\n, \t, \f, \r), refer [link](https://developers.google.com/edu/python/regular-expressions)

5.   Ignore words whose length is less than 3 (Eg: 'is', 'a').

6.   Remove punctuation and non-alphabetic words

7.   Convert the text to lowercase

8.   Use the Porter Stemmer to normalize the words


Refer [link](https://www.nltk.org/book/ch03.html) for extracting the words.

Refer [link](https://medium.com/free-code-camp/an-introduction-to-bag-of-words-and-how-to-code-it-in-python-for-nlp-282e87a9da04) for more information.

In [ ]:
stoplist = set(stopwords.words('english'))
porter = nltk.PorterStemmer()

In [ ]:
def extract_words(question):
    # YOUR CODE HERE
    # Hint: Extract words for each question using the above 8 instructions.
    words = word_tokenize(question)
      
    preprocessed_text = [w for w in words if w not in stoplist] 
    preprocessed_text = [w for w in preprocessed_text if len(w) > 3]
    preprocessed_text = ['name' if w[1]=='NNP' else w[0] for w in  nltk.pos_tag(preprocessed_text)]
    preprocessed_text = [re.sub('\s', " ", w) for w in preprocessed_text]
    remove_punctuation = str.maketrans('', '', string.punctuation)
    preprocessed_text = [w.translate(remove_punctuation).lower() for w in preprocessed_text]
    preprocessed_text = [porter.stem(w) for w in preprocessed_text]
    return preprocessed_text 

In [ ]:
def tokenize(allquestions):
  valid_words = []
  for question in allquestions:
    words = extract_words(question)
    valid_words.extend(words)
  return set(valid_words)

In [ ]:
# Use the function to extract words for all questions
# YOUR CODE HERE

allquestions = features["Questions"].values
vocab = tokenize(allquestions)
len(vocab)

### 4 Marks -> b) Generate vectors that can be used by the machine learning algorithm


1.   The length of the vector for each question will be the length of the valid words. Initialize each vector with all Zeros

2.   Compare each valid word with the words in question and generate the vectors based on the counter frequency of the word in that question.



In [ ]:
def generate_vectors(question):
    # YOUR CODE HERE
    # Hint: Initialize each vector with all zeros. 
    bag_vector = np.zeros(len(vocab))
    
    # Extracting words for each question and count the words
    words = extract_words(question)
    word_dict = Counter(words)
    
    # YOUR CODE HERE 
    # Hint: If the word is in valid words then generate the vectors based on the counter frequency of the word in that question.
    for word in words:
      for i,w in enumerate(vocab):
          if w == word:
              bag_vector[i] = word_dict[word]
    return bag_vector

In [ ]:
# Use the above function for collecting the vectors of all questions into a list.
# YOUR CODE HERE

Sentence_vector = []
for question in allquestions:
  Sentence_vector.append(generate_vectors(question))

## **Stage 4:** Classification

### 3 Marks -> Perform a Classification 

1.   Identify the features and labels

2.   Use train_test_split for splitting the train and test data

3.   Fit your model on the train set using fit() and perform prediction on the test set using predict()

4. Get the accuracy of the model

## Expected Accuracy above 90%


In [ ]:
from sklearn.model_selection import train_test_split

# YOUR CODE HERE
# X is the vectors and Y is the labels
X_train,X_test, y_train, y_test = train_test_split(Sentence_vector,targets,test_size=0.2,random_state=40)

#DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
predicted_labels = clf.predict(X_test)

print(accuracy_score(y_test,predicted_labels))

0.9108910891089109


## **Stage 5:** Evaluation (This is for Mentors)

### 6 Marks -> Evaluate with the given test data 

1.  Loading the Test data

2.  Converting the Test data into vectors

3.  Pass through the model and verify the accuracy

## Expected Accuracy above 90%


In [ ]:
# YOUR CODE HERE for selecting the trained classifier model, eg: MODEL = decision_tree
MODEL = clf #ENTER YOUR MODEL

Test_data = pd.read_csv("Mentors_Test_Data.csv")
Test_data = Test_data[Test_data['Sub-Category'].isin(le.classes_)]
labels = le.transform(Test_data['Sub-Category'])
Test_questions= Test_data['Questions']

Test_BOW=[]
for TQ in Test_questions: 
  Test_vectors = generate_vectors(TQ) 
  Test_BOW.append(Test_vectors)

predict = MODEL.predict(Test_BOW) 
accuracy_score(labels, predict)

0.9113924050632911